# Fuzzy match on address data

The project is to do a fuzzy match on two address files


**Address match process:**

- 1: Data preprocess

   - Capitalize string
      
   - Remove punctuation
      
   - Split street address into street number and street name
   
   
- 2: Data match
 
   - Matched data if same city name
   
   - Do Fuzzy match on street name and filter the result with highest score
   
   - Filter the result if street number is same


- 3:Non_mathed data check

   - Manual check the non_matched data in data source and find the reasons

Let's get started!

## Reference

- Data link:   https://dedupe.io/tutorial/merging-and-matching-multiple-datasets.html


- Compare time on remove punctuation : https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string


## 1: Data import

In [1]:
import numpy as np
import pandas as pd

In [2]:
resturant=pd.read_csv("/Users/admin/Desktop/Address_macth/restaurant-1.csv")
data=pd.read_csv("/Users/admin/Desktop/Address_macth/restaurant-2.csv")

In [3]:
resturant.head()

,name,address,city,cuisine,unique_id
0,arnie morton's of chicago,"""435 s. la cienega blvd.""","""los angeles""","""steakhouses""",'0'
1,art's deli,"""12224 ventura blvd.""","""studio city""","""delis""",'1'
2,bel-air hotel,"""701 stone canyon rd.""","""bel air""","""californian""",'2'
3,cafe bizou,"""14016 ventura blvd.""","""sherman oaks""","""french bistro""",'3'
4,campanile,"""624 s. la brea ave.""","""los angeles""","""californian""",'4'


In [4]:
data.head()

,name_2,address_2,city_2,cuisine_2,unique_id
0,arnie morton's of chicago,"""435 s. la cienega blv.""","""los angeles""","""american""",'0'
1,art's delicatessen,"""12224 ventura blvd.""","""studio city""","""american""",'1'
2,hotel bel-air,"""701 stone canyon rd.""","""bel air""","""californian""",'2'
3,cafe bizou,"""14016 ventura blvd.""","""sherman oaks""","""french""",'3'
4,campanile,"""624 s. la brea ave.""","""los angeles""","""american""",'4'


## 2. Data preprocess

### 2.1 Capitalize address

- Use str.upper() to capitalize data


In [5]:
resturant=resturant.apply(lambda x: x.str.upper())
resturant.head()

,name,address,city,cuisine,unique_id
0,ARNIE MORTON'S OF CHICAGO,"""435 S. LA CIENEGA BLVD.""","""LOS ANGELES""","""STEAKHOUSES""",'0'
1,ART'S DELI,"""12224 VENTURA BLVD.""","""STUDIO CITY""","""DELIS""",'1'
2,BEL-AIR HOTEL,"""701 STONE CANYON RD.""","""BEL AIR""","""CALIFORNIAN""",'2'
3,CAFE BIZOU,"""14016 VENTURA BLVD.""","""SHERMAN OAKS""","""FRENCH BISTRO""",'3'
4,CAMPANILE,"""624 S. LA BREA AVE.""","""LOS ANGELES""","""CALIFORNIAN""",'4'


In [6]:
data=data.apply(lambda x: x.str.upper())
data.head()

,name_2,address_2,city_2,cuisine_2,unique_id
0,ARNIE MORTON'S OF CHICAGO,"""435 S. LA CIENEGA BLV.""","""LOS ANGELES""","""AMERICAN""",'0'
1,ART'S DELICATESSEN,"""12224 VENTURA BLVD.""","""STUDIO CITY""","""AMERICAN""",'1'
2,HOTEL BEL-AIR,"""701 STONE CANYON RD.""","""BEL AIR""","""CALIFORNIAN""",'2'
3,CAFE BIZOU,"""14016 VENTURA BLVD.""","""SHERMAN OAKS""","""FRENCH""",'3'
4,CAMPANILE,"""624 S. LA BREA AVE.""","""LOS ANGELES""","""AMERICAN""",'4'


### 2.2 Remove punctuation

- str.translate(str.maketrans(A,B,C)): copy the string and replace A to B from the maketrans table and delete C;

   - for example: str(translate(str.maketrans('','',string.punctuation)) : replace noting to nothing and remove punctuation which is to copy the whoel strings and delete punctuations in strings

In [7]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
resturant=resturant.apply(lambda x: x.str.translate(str.maketrans('','',string.punctuation)))
resturant.head()

,name,address,city,cuisine,unique_id
0,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0
1,ARTS DELI,12224 VENTURA BLVD,STUDIO CITY,DELIS,1
2,BELAIR HOTEL,701 STONE CANYON RD,BEL AIR,CALIFORNIAN,2
3,CAFE BIZOU,14016 VENTURA BLVD,SHERMAN OAKS,FRENCH BISTRO,3
4,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,CALIFORNIAN,4


In [9]:
data=data.apply(lambda x: x.str.translate(str.maketrans('','',string.punctuation)))
data.head()

,name_2,address_2,city_2,cuisine_2,unique_id
0,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLV,LOS ANGELES,AMERICAN,0
1,ARTS DELICATESSEN,12224 VENTURA BLVD,STUDIO CITY,AMERICAN,1
2,HOTEL BELAIR,701 STONE CANYON RD,BEL AIR,CALIFORNIAN,2
3,CAFE BIZOU,14016 VENTURA BLVD,SHERMAN OAKS,FRENCH,3
4,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,AMERICAN,4


### 2.3 Split column 
Goal: Split full street name to street number and street name to match separtely, since the same locations may have same street name while different street number sometimes from different 
data source.

- series.str.split(pat = None,n=-1,expand= False): 
   - pat is the pattern to split. n is the number of split result. expand= True to expand the list to separate columns
   
   - Conditional split: x.apply(lambda x: x.split() if condition else 0)
   
   - If split a list return NAN: it may because the list been split already
   

- series.replace(A,B): repalce A to B; 
   - if apply to column in dataframe A: use A.column.str.replace (same for all the str methods)
   

- series.str.isdigit(): return True or False wether the string is digit


- series.apply(pd.Series): change series type to DataFrame


- regex: re.match == str.match; use ^[0-9] : define string starts with digit


- get the first character of a column: resturant.address.str[0]; while if it is value of address, then no 'str' need: resturant.address[6][0]

In [10]:
## Can't split more, since [A,B] is the result after split
resturant.address.apply(lambda x: x.split(' ',2)[1:3] if x.find('[0-9]') else [0,x]).head()

0    [435, S LA CIENEGA BLVD]
1       [12224, VENTURA BLVD]
2      [701, STONE CANYON RD]
3       [14016, VENTURA BLVD]
4        [624, S LA BREA AVE]
Name: address, dtype: object

In [11]:
# split column into number and name if column startwith digit
resturant[['address_no','address_name']]=resturant.address.apply(lambda x: x.split(' ',2)[1:3] if x.replace(' ','')[0].isdigit() else [0,x]).apply(pd.Series)

In [12]:
resturant.head()

,name,address,city,cuisine,unique_id,address_no,address_name
0,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD
1,ARTS DELI,12224 VENTURA BLVD,STUDIO CITY,DELIS,1,12224,VENTURA BLVD
2,BELAIR HOTEL,701 STONE CANYON RD,BEL AIR,CALIFORNIAN,2,701,STONE CANYON RD
3,CAFE BIZOU,14016 VENTURA BLVD,SHERMAN OAKS,FRENCH BISTRO,3,14016,VENTURA BLVD
4,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,CALIFORNIAN,4,624,S LA BREA AVE


In [13]:
data[['address_no','address_name']]=data.address_2.apply(lambda x: x.split(' ',2)[1:3] if x.replace(' ','')[0].isdigit() else [0,x]).apply(pd.Series)

In [14]:
data.head()

,name_2,address_2,city_2,cuisine_2,unique_id,address_no,address_name
0,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLV,LOS ANGELES,AMERICAN,0,435,S LA CIENEGA BLV
1,ARTS DELICATESSEN,12224 VENTURA BLVD,STUDIO CITY,AMERICAN,1,12224,VENTURA BLVD
2,HOTEL BELAIR,701 STONE CANYON RD,BEL AIR,CALIFORNIAN,2,701,STONE CANYON RD
3,CAFE BIZOU,14016 VENTURA BLVD,SHERMAN OAKS,FRENCH,3,14016,VENTURA BLVD
4,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,AMERICAN,4,624,S LA BREA AVE


## 3 Fuzzy match

- fuzz.ratio(A,B): give the fuzz ratio for A and B


- pd.merge(DF1,DF2, left_on='', right_on='', how='') merge two dataframe: how = inner, left, right  OR pd.merge(DF1,DF2, on='', how='') when mrge column names are same
    

- astype(str): change data type


- DF.reset_index(): reset the index to column or flatern the result from groupby

In [15]:
from fuzzywuzzy import fuzz

In [16]:
# match if the city name is same
result=pd.merge(resturant, data, left_on='city',right_on='city_2',how='left').apply(lambda x: x.astype(str))
result.head()

,name,address,city,cuisine,unique_id_x,address_no_x,address_name_x,name_2,address_2,city_2,cuisine_2,unique_id_y,address_no_y,address_name_y
0,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLV,LOS ANGELES,AMERICAN,0,435,S LA CIENEGA BLV
1,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,AMERICAN,4,624,S LA BREA AVE
2,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,CITRUS,6703 MELROSE AVE,LOS ANGELES,CALIFORNIAN,6,6703,MELROSE AVE
3,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,GRILL ON THE ALLEY,9560 DAYTON WAY,LOS ANGELES,AMERICAN,9,9560,DAYTON WAY
4,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,RESTAURANT KATSU,1972 N HILLHURST AVE,LOS ANGELES,ASIAN,10,1972,N HILLHURST AVE


In [17]:
# calculate the fuzz ratio
result['address_name_score']=result.apply(lambda x: fuzz.ratio(x.address_name_x,x.address_name_y),axis=1)
result.head()

,name,address,city,cuisine,unique_id_x,address_no_x,address_name_x,name_2,address_2,city_2,cuisine_2,unique_id_y,address_no_y,address_name_y,address_name_score
0,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLV,LOS ANGELES,AMERICAN,0,435,S LA CIENEGA BLV,97
1,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,AMERICAN,4,624,S LA BREA AVE,60
2,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,CITRUS,6703 MELROSE AVE,LOS ANGELES,CALIFORNIAN,6,6703,MELROSE AVE,29
3,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,GRILL ON THE ALLEY,9560 DAYTON WAY,LOS ANGELES,AMERICAN,9,9560,DAYTON WAY,22
4,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,RESTAURANT KATSU,1972 N HILLHURST AVE,LOS ANGELES,ASIAN,10,1972,N HILLHURST AVE,31


In [18]:
## learned: reset_index(): reset the index as one column
## get the max score per name
max_address_name_score=result.groupby('name').address_name_score.max().reset_index().apply(pd.Series)
max_address_name_score.head()

,name,address_name_score
0,21 CLUB,89
1,ABRUZZI,100
2,ALAIN RONDELLI,100
3,AQUA,100
4,AQUAVIT,89


In [19]:
## filter the highest score data
result1=pd.merge(result,max_address_name_score,on=['name','address_name_score'],how='inner')

In [20]:
# filter data if address number is same
result2=result1[result1['address_no_x']==result1['address_no_y']]
result2.head()

,name,address,city,cuisine,unique_id_x,address_no_x,address_name_x,name_2,address_2,city_2,cuisine_2,unique_id_y,address_no_y,address_name_y,address_name_score
0,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLVD,LOS ANGELES,STEAKHOUSES,0,435,S LA CIENEGA BLVD,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLV,LOS ANGELES,AMERICAN,0,435,S LA CIENEGA BLV,97
1,ARTS DELI,12224 VENTURA BLVD,STUDIO CITY,DELIS,1,12224,VENTURA BLVD,ARTS DELICATESSEN,12224 VENTURA BLVD,STUDIO CITY,AMERICAN,1,12224,VENTURA BLVD,100
3,BELAIR HOTEL,701 STONE CANYON RD,BEL AIR,CALIFORNIAN,2,701,STONE CANYON RD,HOTEL BELAIR,701 STONE CANYON RD,BEL AIR,CALIFORNIAN,2,701,STONE CANYON RD,100
4,CAFE BIZOU,14016 VENTURA BLVD,SHERMAN OAKS,FRENCH BISTRO,3,14016,VENTURA BLVD,CAFE BIZOU,14016 VENTURA BLVD,SHERMAN OAKS,FRENCH,3,14016,VENTURA BLVD,100
6,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,CALIFORNIAN,4,624,S LA BREA AVE,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,AMERICAN,4,624,S LA BREA AVE,100


## 4 Check on the non-macthed

- DF.column.isin(array): get data if the column value in arrays

In [21]:
name_list=resturant.name.unique()
matched_names=result2.name.unique()

In [22]:
non_matched=[i for i in name_list if i not in matched_names]

In [27]:
# Summary of the fuzzy match
print('Total distinct names:{}, number of names matched: {},number of names not_matched {}'.format(len(name_list),len(matched_names),len(non_matched)))

Total distinct names:112, number of names matched: 47,number of names not_matched 65


In [30]:
non_matched_data=resturant[resturant['name'].isin(non_matched)]
non_matched_data.head()

,name,address,city,cuisine,unique_id,address_no,address_name
7,FENIX AT THE ARGYLE,8358 SUNSET BLVD,W HOLLYWOOD,FRENCH NEW,7,8358,SUNSET BLVD
9,GRILL THE,9560 DAYTON WAY,BEVERLY HILLS,AMERICAN TRADITIONAL,9,9560,DAYTON WAY
10,KATSU,1972 HILLHURST AVE,LOS FELIZ,JAPANESE,10,1972,HILLHURST AVE
11,LORANGERIE,903 N LA CIENEGA BLVD,W HOLLYWOOD,FRENCH CLASSIC,11,903,N LA CIENEGA BLVD
13,LOCANDA VENETA,8638 W THIRD ST,LOS ANGELES,ITALIAN,13,8638,W THIRD ST


In [31]:
data.head()

,name_2,address_2,city_2,cuisine_2,unique_id,address_no,address_name
0,ARNIE MORTONS OF CHICAGO,435 S LA CIENEGA BLV,LOS ANGELES,AMERICAN,0,435,S LA CIENEGA BLV
1,ARTS DELICATESSEN,12224 VENTURA BLVD,STUDIO CITY,AMERICAN,1,12224,VENTURA BLVD
2,HOTEL BELAIR,701 STONE CANYON RD,BEL AIR,CALIFORNIAN,2,701,STONE CANYON RD
3,CAFE BIZOU,14016 VENTURA BLVD,SHERMAN OAKS,FRENCH,3,14016,VENTURA BLVD
4,CAMPANILE,624 S LA BREA AVE,LOS ANGELES,AMERICAN,4,624,S LA BREA AVE


In [32]:
# use contains to manual check if data contains same string
data[data.address_2.str.contains('SUNSET')]

,name_2,address_2,city_2,cuisine_2,unique_id,address_no,address_name
7,FENIX,8358 SUNSET BLVD WEST,HOLLYWOOD,AMERICAN,7,8358,SUNSET BLVD WEST
136,GLADSTONES,4 FISH 17300 PACIFIC COAST HWY AT SUNSET BLVD,PACIFIC PALISADES,AMERICAN,136,4,FISH 17300 PACIFIC COAST HWY AT SUNSET BLVD
143,JOSS,9255 SUNSET BLVD,LOS ANGELES,ASIAN,143,9255,SUNSET BLVD
145,LE DOME,8720 SUNSET BLVD,LOS ANGELES,FRENCH,145,8720,SUNSET BLVD
550,DUKES,8909 SUNSET BLVD,W HOLLYWOOD,COFFEE SHOPS,551,8909,SUNSET BLVD
